In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import glob

In [2]:
positions = ["PosX", "PosY", "PosZ"]
rotations = ["RotX", "RotY", "RotZ"]
tmp = positions + rotations


path = r'../from_study/CSV' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, delimiter=";")
    li.append(df)


def calc_euc(df):
    for pos in positions:
        df[pos + "_diff"] = df[pos] - df[pos].shift(1)
        df[pos + "_square"] = df[pos + "_diff"] * df[pos + "_diff"]

    df["euc"] = (df["PosX_square"] + df["PosY_square"] + df["PosZ_square"]).apply(lambda x: np.sqrt(x))
    return df["euc"].sum()

In [3]:
df_result = pd.DataFrame(columns=['PID','condition','movement'])
for j in range(len(li)):
    df = li[j]
    df = df[(df.isStudy == True)]
    # filter unusual data
    df = df[(df.PosX < 10) & (df.PosX > -10)]
    df = df[(df.PosY < 10) & (df.PosY > -10)]
    df = df[(df.PosZ < 10) & (df.PosZ > -10)]
    for i in range(1, 4):
        right = df[(df.Hand == "Right") & (df.Condition == i)]
        left = df[(df.Hand == "Left") & (df.Condition == i)]
        tmp = calc_euc(right) + calc_euc(left)
        pid = df.SubjectID.unique()[0]
        df_result = df_result.append({'PID': pid, 'condition': i, 'movement': tmp}, ignore_index=True)
        # print("PID", pid, "-", "condition", str(i), ":\t", tmp)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
df_result["PID"] = pd.to_numeric(df_result["PID"]).astype(int)
df_result["condition"] = pd.to_numeric(df_result["condition"]).astype(int)

In [5]:
df_result.groupby(["condition"]).movement.describe()

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
1,12.0,71.271603,33.819030,13.868938,54.588462,65.702882,86.344528,140.308256
2,12.0,64.517381,34.622370,21.475518,43.877621,53.898466,81.488739,140.276817
3,12.0,67.556325,26.522171,27.787852,49.864273,63.915033,73.766878,120.113180


presence ist gleich wie bei condtions

eigene presence ist daher nicht beeinflusst